In [1]:
!pip -q install sweetviz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 37.4 MB/s eta 0:00:0000:0100:01


In [6]:
# ---------- Robust loader for marketing_campaign.csv ----------
import os
import pandas as pd

CSV_PATHS = [
    "/kaggle/input/customer-personality-analysis/marketing_campaign.csv",
    "/kaggle/input/customer-personality-analysis/MarketingCampaign.csv",
    "data/marketing_campaign.csv",
    "marketing_campaign.csv",
]

def load_marketing(paths):
    last_err = None
    for p in paths:
        if not os.path.exists(p):
            continue
        try:
            # 1) Try automatic delimiter inference
            df = pd.read_csv(p, sep=None, engine="python")
            if df.shape[1] == 1:
                # 2) Fall back through common delimiters
                for sep in [";", "\t", ",", "|"]:
                    df2 = pd.read_csv(p, sep=sep)
                    if df2.shape[1] >= 10:  # expect many columns
                        df = df2
                        break
            print(f"[OK] Loaded with shape {df.shape} from {p}")
            return df, p
        except Exception as e:
            last_err = e
    raise RuntimeError(f"Failed to load from {paths}. Last error: {last_err}")

df, used_path = load_marketing(CSV_PATHS)

# ---------- Normalize column names ----------
df.columns = [str(c).strip() for c in df.columns]
print("Parsed columns:", df.columns.tolist())

# Quick sanity: ensure 'Response' exists & is numeric
if "Response" not in df.columns:
    raise ValueError("Still missing 'Response' after reparse. Check delimiter/encoding.")
df["Response"] = pd.to_numeric(df["Response"], errors="coerce").fillna(0).astype(int)

# (Optional) Useful type fixes
date_col = "Dt_Customer" if "Dt_Customer" in df.columns else None
if date_col:
    df[date_col] = pd.to_datetime(df[date_col], errors="coerce", dayfirst=True)

# ---------- Re-run SweetViz reports ----------
import sweetviz as sv
REPORT_DIR = "Reports"
os.makedirs(REPORT_DIR, exist_ok=True)

# Overall
report_overall = sv.analyze(df)
report_overall.show_html(os.path.join(REPORT_DIR, "sweetviz_overall.html"))
print(f"[OK] Saved: {os.path.join(REPORT_DIR, 'sweetviz_overall.html')}")

# Target report using the REAL 'Response'
report_target = sv.analyze([df, "Marketing"], target_feat="Response")
report_target.show_html(os.path.join(REPORT_DIR, "sweetviz_target_Response.html"))
print(f"[OK] Saved: {os.path.join(REPORT_DIR, 'sweetviz_target_Response.html')}")


[OK] Loaded with shape (2240, 29) from /kaggle/input/customer-personality-analysis/marketing_campaign.csv
Parsed columns: ['ID', 'Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome', 'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1', 'AcceptedCmp2', 'Complain', 'Z_CostContact', 'Z_Revenue', 'Response']


                                             |          | [  0%]   00:00 -> (? left)

Report Reports/sweetviz_overall.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
[OK] Saved: Reports/sweetviz_overall.html


                                             |          | [  0%]   00:00 -> (? left)

Report Reports/sweetviz_target_Response.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
[OK] Saved: Reports/sweetviz_target_Response.html
